# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

Execution Date/time is 27.01.2022 15:46:30


In [ ]:
//#r "BoSSSpad.dll"
#r "B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
using BoSSS.Application.SipPoisson;

In [ ]:
ExecutionQueues

index,type,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,MonoDebug,AllowedDatabasesPaths,BatchInstructionDir,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=2000 ]",/home/jw52xeqa/Deployerie,special00006,<null>,X:\Deployerie,True,HHLR,dotnet,False,[ W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa ],,,,
1,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,,,,,,,,,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,,[ D:\trash_db == ],<null>,,,
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,FDY\weber,,DC2,,,,,,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,[ \\hpccluster\hpccluster-scratch\weber\trash_DB == ],,[ HPCCLUSTER2 ],Normal,True
3,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,X:\Deployerie,True,<null>,dotnet,False,,,,,


In [ ]:
var myBatch = ExecutionQueues[2];
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,FDY\weber,DC2,[ HPCCLUSTER2 ],Normal,True,[ \\hpccluster\hpccluster-scratch\weber\trash_DB == ]


In [ ]:
wmg.Init("CCPoisson_conditionNo");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.DefaultDatabase = OpenOrCreateDatabase(ExecutionQueues[2].AllowedDatabasesPaths.Pick(0).LocalMountPath);

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [ ]:
wmg.Grids

#0: { Guid = 47fa58f5-ab4d-4236-9207-1313c8fbea12; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }
#1: { Guid = 766260bb-b87a-4de8-9716-7cc5cd3b59a3; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
#2: { Guid = 8e1e5757-aab8-45c7-8ca8-83261f901f04; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
#3: { Guid = 74db1319-7ecf-47df-91e1-c86f9f47e2a3; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = bc406965-9bcc-4ca5-bacc-81b044a3f272; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
#5: { Guid = c950736e-7cd6-42db-992a-e0b5517b52f2; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
#6: { Guid = 88fe21a5-09a9-4d8d-8ada-435d6247b5d0; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }
#7: { Guid = ec1d8aaf-3737-45cd-8065-283b1a0cdf77; Name = RotBenchmarkGrid; Cell Count = 8232; Dim = 3 }
#8: { Guid = cdd10f9f-afc

Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [ ]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
grids

#0: { Guid = 88fe21a5-09a9-4d8d-8ada-435d6247b5d0; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }
#1: { Guid = c950736e-7cd6-42db-992a-e0b5517b52f2; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
#2: { Guid = bc406965-9bcc-4ca5-bacc-81b044a3f272; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
#3: { Guid = 74db1319-7ecf-47df-91e1-c86f9f47e2a3; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = 8e1e5757-aab8-45c7-8ca8-83261f901f04; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
#5: { Guid = 766260bb-b87a-4de8-9716-7cc5cd3b59a3; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
#6: { Guid = 47fa58f5-ab4d-4236-9207-1313c8fbea12; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }


## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [ ]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [ ]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.19866933079506122

In [ ]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.8390715290764524

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    //LinearSolverCode.exp_Kcycle_schwarz,
    //LinearSolverCode.exp_gmres_levelpmg,
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 5000000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
List<SipControl> controls = new List<SipControl>();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name)) {
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add(ctrl);
       
    
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.SetDatabase(wmg.DefaultDatabase);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver.SolverCode           = solver_name;
    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.PostprocessingModules.Add(new CondLogger());
    
    ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}

setting up: J320_k2_classic_pardiso
setting up: J2560_k2_classic_pardiso
setting up: J20480_k2_classic_pardiso
setting up: J69120_k2_classic_pardiso
setting up: J163840_k2_classic_pardiso
setting up: J320_k3_classic_pardiso
setting up: J2560_k3_classic_pardiso
setting up: J20480_k3_classic_pardiso
setting up: J69120_k3_classic_pardiso
setting up: J163840_k3_classic_pardiso
setting up: J320_k5_classic_pardiso
setting up: J2560_k5_classic_pardiso
setting up: J20480_k5_classic_pardiso
setting up: J69120_k5_classic_pardiso


## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    ctrl.SessionName += "";
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(myBatch); 
}

 Submitting: SIP_J320_k2_classic_pardiso
Deploying job SIP_J320_k2_classic_pardiso ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\weber\dply\CCPoisson_conditionNo-ipPoisson2022Jan27_154641
copied 42 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.

 Submitting: SIP_J2560_k2_classic_pardiso
Deploying job SIP_J2560_k2_classic_pardiso ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\weber\dply\CCPoisson_conditionNo-ipPoisson2022Jan27_154646
copied 42 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
Loading session 433e6436-0c5e-4ce9-bc50-4c43ed3133cd failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\weber\trash_DB\sessions\433e6436-0c5e-4ce9-bc50-4c43ed3133cd\Session.info'.'

 Submitting: SIP_J20480_k2_classic_pardiso
Loading session 433e6436-0c5e-4ce9-bc50-4c43ed3133cd failed with 

In [ ]:
//foreach(var ctrl in controls) {
//    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
//    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
//    aJob.SetControlObject(ctrl);
//    ((SlurmClient)myBatch).SlurmAccount = "project01217";  // Jens 
//    aJob.NumberOfMPIProcs = 1;
//    aJob.ExecutionTime    = "02:00:00";
//    //aJob.HHLR_project             = "project01217";
//    aJob.MemPerCPU                = "50000";
//    aJob.UseComputeNodesExclusive = true;
//    aJob.Activate(myBatch);
//}

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful);
NoSuccess

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in wmg.AllJobs.Values)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => (Si.SuccessfulTermination == false)
                                        || (Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail.LatestDeployment.DeploymentDirectory.Name);

## Create Table for Post-Processing

(moved to post-processing worksheet)

In [ ]:
/*
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}
*/

In [ ]:
/*
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
*/

In [ ]:
/*
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());
*/

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
/*var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");
*/

In [ ]:
//SubTab.Print();

In [ ]:
/*
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");
*/